In [ ]:
import train
import numpy as np
import api.capital_features_api as cf
import pandas as pd
import mplfinance as mpf
from datetime import datetime
import joblib
import matplotlib.pyplot as plt

In [ ]:
train_up_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1070492)
validation_up_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1070494)
train_down_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1070493)
validation_down_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1070495)

In [ ]:
Y_up_train = train_up_data['upSignals']
Y_up_val = validation_up_data['upSignals']
Y_down_train = train_down_data['downSignals']
Y_down_val = validation_down_data['downSignals']

In [ ]:
train_up_dataset, validation_up_dataset, scaler_up = train.to_dataset(train_up_data, validation_up_data, Y_up_train, Y_up_val)
train_down_dataset, validation_down_dataset, scaler_down = train.to_dataset(train_down_data, validation_down_data, Y_down_train, Y_down_val)
weights_up = train.to_weights(Y_up_train)
weights_down = train.to_weights(Y_down_train)

In [ ]:
model_up, callbacks_up = train.get_model(5, 40, 1e-1)
model_down, callbacks_down = train.get_model(5, 40, 1e-1)

In [ ]:
his_up = model_up.fit(train_up_dataset, epochs=100, validation_data=validation_up_dataset, verbose=1, callbacks=callbacks_up,
                   class_weight=weights_up)

In [ ]:
his_down = model_down.fit(train_down_dataset, epochs=100, validation_data=validation_down_dataset, verbose=1, callbacks=callbacks_down,
                   class_weight=weights_down)

In [ ]:
plt.plot(his_up.history['accuracy'], label='up_accuracy')
plt.plot(his_up.history['val_accuracy'], label='up_val_accuracy')
plt.plot(his_down.history['accuracy'], label='down_accuracy')
plt.plot(his_down.history['val_accuracy'], label='down_val_accuracy')
plt.legend()
plt.show()

In [ ]:
model_up.save('./model/spy500_model_up_40_ty.h5')
model_down.save('./model/spy500_model_down_40_ty.h5')
model_up.save('./model/spy500_model_up_40_ty', save_format='tf')
model_down.save('./model/spy500_model_down_40_ty', save_format='tf')
joblib.dump(scaler_up, './model/spy500_model_up_40_ty_scaler_up.save')
joblib.dump(scaler_down, './model/spy500_model_down_40_ty_scaler_down.save')
train.save_scaler('./model/spy500_model_up_40_ty_scaler_up.json', scaler_up)
train.save_scaler('./model/spy500_model_down_40_ty_scaler_down.json', scaler_down)

In [ ]:
prediction_data = cf.get_prediction_data(cf.docker_prediction_data_url, train_config_id=7, start="2019-07-11", end=datetime.now().strftime("%Y-%m-%d"), code='QQQ')

In [ ]:
X_prediction_scaled = train.to_prediction_scaled(prediction_data, scaler_up)
Y_up_prediction = model_up.predict(X_prediction_scaled)
Y_down_prediction = model_down.predict(X_prediction_scaled)

In [ ]:
kline = cf.get_kline_histroy(cf.docker_kline_history_url, code="QQQ", start="2019-07-11", end=datetime.now().strftime("%Y-%m-%d"))
df = cf.his_to_df(kline)

In [ ]:
predictions_up_series = pd.Series(Y_up_prediction.flatten(), index=df.index)
buy_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
buy_signals[predictions_up_series > 0.7] = df['low'][predictions_up_series > 0.7] * 0.99
buy_markers = mpf.make_addplot(buy_signals, type='scatter', markersize=100, marker='^', color='g')

predictions_down_series = pd.Series(Y_down_prediction.flatten(), index=df.index)
sell_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
sell_signals[predictions_down_series > 0.7] = df['high'][predictions_down_series > 0.7] * 1.01
sell_markers = mpf.make_addplot(sell_signals, type='scatter', markersize=100, marker='o', color='r')

mpf.plot(df, type='candle', style='charles', addplot=[buy_markers, sell_markers], volume=True, figsize=(24, 16))